In [ ]:
from qiskit import Aer, QuantumCircuit, transpile, assemble
from qiskit.visualization import plot_histogram

def initialize_s(qc, qubits):
    """Apply a Hadamard gate to all qubits in the list."""
    for q in qubits:
        qc.h(q)

def oracle(qc, target_qubit): #bad 
    """Apply a phase oracle to the target qubit."""
    qc.z(target_qubit)

def diffusion(qc, qubits):
    """Apply the diffusion operator to the qubits."""
    # Apply Hadamard gates
    qc.h(qubits)
    # Apply X gates
    qc.x(qubits)
    # Apply controlled-Z gate
    qc.h(qubits[-1])
    qc.mct(qubits[:-1], qubits[-1])
    # Apply X gates
    qc.x(qubits)
    # Apply Hadamard gates
    qc.h(qubits)

def grovers_algorithm(n, marked_items):
    # Create a quantum circuit with n qubits and n classical bits
    qc = QuantumCircuit(n, n)
    
    # Initialize the superposition of all states
    initialize_s(qc, range(n))
    
    # Define the number of Grover iterations (adjust as needed)
    num_iterations = int(0.5 * (2 ** (n / 2)))
    
    # Apply Grover iterations
    for _ in range(num_iterations):
        # Apply the oracle to mark the solution(s)
        oracle(qc, marked_items)
        
        # Apply the diffusion operator
        diffusion(qc, range(n))
    
    # Measure the qubits
    qc.measure(range(n), range(n))
    
    # Simulate the quantum circuit
    simulator = Aer.get_backend('qasm_simulator')
    compiled_circuit = transpile(qc, simulator)
    result = simulator.run(compiled_circuit).result()
    
    # Get and plot the result
    counts = result.get_counts(qc)
    plot_histogram(counts)

# Example usage
n_qubits = 3  # Number of qubits
marked_items = [4]  # Indices of marked items

grovers_algorithm(n_qubits, marked_items)